In [1]:
!python -m spacy download en_core_web_md

2023-12-18 09:44:36.882992: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-18 09:44:36.883078: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-18 09:44:36.885642: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-18 09:44:36.901714: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-12-18 09:44:41.078567: W tensorflow/compiler/tf2

In [2]:
import spacy
import numpy as np
import csv
nlp = spacy.load("en_core_web_md")

In [7]:
categories = {
    'General Education': ['English Composition & Comprehension','English Composition & Comprehension Lab', 'Technical & Business Writing',
                          'Communication & Presentation Skills', 'Professional Practices',
                          'Introduction to Info. & Comm. Technologies', 'Pakistan Studies', 'Islamic Studies & Ethics','Ethics'],
    'University Electives': ['Foreign Language', 'Social Service', 'Management Related',
                             'Social Science Related', 'Economy Related'],
    'Mathematics & Science Foundation': ['Calculus & Analytical Geometry', 'Probability & Statistics',
                                         'Linear Algebra', 'Applied Physics'],
    'Computing – Core': ['Programming Fundamentals','Programming Fundamentals Lab', 'Object Oriented Programming','Object Oriented Programming Lab', 'Data Structures & Algorithms',
                         'Data Structures & Algorithms Lab','Operating Systems Lab','Database Systems Lab','Computer Networks Lab',
                         'Discrete Structures', 'Operating Systems', 'Database Systems', 'Software Engineering',
                         'Computer Networks', 'Information Security', 'Final Year Project'],
    'Domain CS Core': ['Compiler Construction', 'Comp. Organization & Assembly Language','Comp. Organization & Assembly Language Lab', 'Digital Logic Design',
                       'Digital Logic Design','Artificial Intelligence Lab',
                       'Design & Analysis of Algorithms', 'Parallel & Distributed Computing',
                       'Artificial Intelligence', 'Theory of Automata','Domain CS Core','Domain CS Core Lab'],
    'Domain CS Supporting': ['Differential Equations', 'Multi-variate Calculus', 'Graph Theory',
                             'Theory of Programming Languages', 'Numerical Computing','Domain CS Supporting'],
    'Domain CS Electives': ['CS Elective - 1','CS Elective - 2','CS Elective - 3','CS Elective - 4','CS Elective - 5','CS Electives']
}

# Function to calculate average word vector for a text
def get_average_vector(text):
    vectors = [word.vector for word in nlp(text) if word.has_vector]
    if not vectors:
        return None
    return sum(vectors) / len(vectors)

# Function to categorize a course
def categorize_course(course):
    course_vector = get_average_vector(course.lower())

    max_similarity = 0
    best_category = "Uncategorized"

    if course_vector is not None:
        for category, courses in categories.items():
            for predefined_course in courses:
                predefined_vector = get_average_vector(predefined_course.lower())
                if predefined_vector is not None:
                    similarity = course_vector.dot(predefined_vector) / (np.linalg.norm(course_vector) * np.linalg.norm(predefined_vector))

                    if similarity > max_similarity:
                        max_similarity = similarity
                        best_category = category


    return best_category if max_similarity > 0.5 else "Uncategorized"

# Function to calculate credit hours for a specific category
def calculate_credit_hours(transcript, category_courses):
    total_credit_hours = 0

    for course in transcript:
        if course['courseName'] in category_courses:
            if 'points' in course:
                points = float(course['points'])
                if points >= 1:
                    credit_hours = int(course['creditHour'])
                    total_credit_hours += credit_hours
                elif points < 1:
                  fail_list.append(course['courseName'])
    return total_credit_hours

def create_category_course_dict(data):
    category_course_dict = {}

    for entry in data:
        category = entry['category']
        course_name = entry['courseName']

        if category not in category_course_dict:
            category_course_dict[category] = []

        category_course_dict[category].append(course_name)

    return category_course_dict

apni_list=[]
fail_list = []

def filter_records(data):
    rmk_records = [record for record in data if record.get('rmk') == 'R-1']
    rmk_course_names = {rmk_record.get('courseName') for rmk_record in rmk_records}
    filtered_data = [
        record for record in data
        if record.get('courseName') not in rmk_course_names or record.get('rmk') == 'R-1'
    ]
    return filtered_data


def filter_records_test(data):
    zero_points_records = [record for record in data if float(record.get('points', 0)) <= 0]
    zero_points_course_names = {record.get('courseName') for record in zero_points_records}
    filtered_data = [
        record for record in data
        if float(record.get('points', 0)) > 0 or record.get('courseName') not in zero_points_course_names
    ]
    return filtered_data



# Function to categorize courses in the transcript
def categorize_transcript_courses(transcript):
    categorized_transcript = []

    for course in transcript:
        course_name = course['courseName']
        category = categorize_course(course_name)
        apni_list.append(course_name)
        course['category'] = category
        categorized_transcript.append(course)

    return categorized_transcript

# Function to check degree completion by category
def check_degree_completion(transcript):
    categorized_transcript = categorize_transcript_courses(transcript)

    for course in categorized_transcript:
      if course['category'] == 'Mathematics & Science Foundation':
          print(f"Course Name: {course['courseName']}","  ",course["creditHour"],"  ",course["points"])

    required_credit_hours = {
        'General Education': 19,
        'University Electives': 12,
        'Mathematics & Science Foundation': 12,
        'Computing – Core': 39,
        'Domain CS Core': 24,
        'Domain CS Electives': 15,
        'Domain CS Supporting': 9
    }

    category_courses = {
        'General Education': categories['General Education'],
        'University Electives': categories['University Electives'],
        'Mathematics & Science Foundation': categories['Mathematics & Science Foundation'],
        'Computing – Core': categories['Computing – Core'],
        'Domain CS Core': categories['Domain CS Core'],
        'Domain CS Electives': categories['Domain CS Electives'],
        'Domain CS Supporting': categories['Domain CS Supporting']
    }
    global result
    result = create_category_course_dict(categorized_transcript)
    category_credit_hours = {}
    for category, courses in result.items():
        category_credit_hours[category] = calculate_credit_hours(categorized_transcript,courses)

    global C_creds
    global R_creds
    C_creds={}
    R_creds={}
    C_hrs=0
    R_hrs=0
    print("All Categories With Total Credit Hrs:")
    print('-------------------------------------\n\n')
    for category, required_hours in required_credit_hours.items():
        completed_hours = category_credit_hours.get(category, 0)
        if completed_hours>required_hours:
          C_hrs=C_hrs+required_hours
        else:
          C_hrs=C_hrs+completed_hours
        R_hrs=R_hrs+required_hours
        C_creds[category]=completed_hours
        R_creds[category]=required_hours
        print(category, '========> Completed', completed_hours, 'out of', required_hours,'\t(',required_hours-completed_hours,'credit hours required!)')

    print('\n------------------------------------')
    print("Your Have Earned ",C_hrs,"Out Of ",R_hrs)
    print('------------------------------------')

# Loading transcript data
with open('Test Std-0102.csv', newline='') as csvfile:
    reader = csv.DictReader(csvfile)
    transcript_data = list(reader)
    filtered_list = filter_records(transcript_data)
    #filtered_list = filter_records_test(transcript_data)

# Check degree completion
check_degree_completion(filtered_list)
print("\n\nCourse That You've Failed:")
print('-------------------------------\n')

if not fail_list:
  print("None")
else:
  for i in fail_list:
    print(i)

print("\n\nCourses That Can Be Picked In Order To FullFill Credit Hrs:")
print('----------------------------------------------------------------\n')
required = {}
for key, value in categories.items():
  required[key] = [course for course in value if course not in apni_list]

for key, value in required.items():
  if C_creds[key]<R_creds[key]:
    print(key, '========>', value)






Course Name: Linear Algebra    3    0
Course Name: Linear Algebra    3    0
Course Name: Linear Algebra    3    3
All Categories With Total Credit Hrs:
-------------------------------------


General Education ========> Completed 19 out of 19 	( 0 credit hours required!)
University Electives ========> Completed 9 out of 12 	( 3 credit hours required!)
Mathematics & Science Foundation ========> Completed 3 out of 12 	( 9 credit hours required!)
Computing – Core ========> Completed 41 out of 39 	( -2 credit hours required!)
Domain CS Core ========> Completed 24 out of 24 	( 0 credit hours required!)
Domain CS Electives ========> Completed 0 out of 15 	( 15 credit hours required!)
Domain CS Supporting ========> Completed 25 out of 9 	( -16 credit hours required!)

------------------------------------
Your Have Earned  103 Out Of  130
------------------------------------


Course That You've Failed:
-------------------------------

Computer Programming
Computer Prog Lab
Discrete Structures